Fine-tuning HateBERT: full-model, 4 epochs with a learning rate of 5e-6 and weight decay of 0.01 and a batch size 32. Training input data were truncated at 512 tokens.

# Install

In [ ]:
!pip install -U -q datasets transformers torchinfo

     |████████████████████████████████| 441 kB 34.5 MB/s 
     |████████████████████████████████| 5.5 MB 52.8 MB/s 
     |████████████████████████████████| 95 kB 3.5 MB/s 
     |████████████████████████████████| 115 kB 49.2 MB/s 
     |████████████████████████████████| 163 kB 8.8 MB/s 
     |████████████████████████████████| 212 kB 52.0 MB/s 
     |████████████████████████████████| 127 kB 59.9 MB/s 
     |████████████████████████████████| 7.6 MB 48.2 MB/s 
     |████████████████████████████████| 115 kB 68.7 MB/s 


# Imports

In [ ]:
from datasets import Dataset, DatasetDict, ClassLabel, Value, load_dataset, load_metric
from huggingface_hub import notebook_login
from torchinfo import summary
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

# Load Data

In [ ]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
ds = load_dataset('sara-nabhani/lfd-proj',
    'csv', 
    data_files={'train': 'train.csv', 'val': 'val.csv', 'test': 'test.csv'}
)

cl = ClassLabel(names=list(ds['train'].unique('label')))
ds = ds.cast_column('label', cl)

ds

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/sara-nabhani___csv/sara-nabhani--lfd-proj-a35fbd5b9bbbc3d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 12240
    })
    val: Dataset({
        features: ['tweet', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 860
    })
})

In [ ]:
ds['train'].features

{'tweet': Value(dtype='string', id=None),
 'label': ClassLabel(names=['OFF', 'NOT'], id=None)}

In [ ]:
ds['test'].features

{'tweet': Value(dtype='string', id=None),
 'label': ClassLabel(names=['OFF', 'NOT'], id=None)}

In [ ]:
ds['val'].features

{'tweet': Value(dtype='string', id=None),
 'label': ClassLabel(names=['OFF', 'NOT'], id=None)}

In [ ]:
import torch
torch.cuda.is_available()

True

# Preprocessing

In [ ]:
model_id = 'GroNLP/hateBERT'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
    return tokenizer(examples['tweet'], padding='max_length', truncation=True)

tokenized_ds = ds.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/151 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# Model loading and training

In [ ]:
metric = load_metric("f1")
# evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='macro')


model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
training_args = TrainingArguments(    
  output_dir="./results",
  report_to='all',
  learning_rate=5e-6,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=32,
  num_train_epochs=4,
  weight_decay=0.01,
  no_cuda=False,
  # bf16=IS_CUDA_AVAILABLE,
  fp16=True,
  evaluation_strategy='epoch',
  logging_strategy='epoch',
)
  
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_ds['train'],
  eval_dataset=tokenized_ds['val'],
  compute_metrics=compute_metrics,
  tokenizer=tokenizer,
  data_collator=data_collator,
)
trainer.train()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at GroNLP/hateBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly

Epoch,Training Loss,Validation Loss,F1
1,0.535900,0.465883,0.766317
2,0.427000,0.439914,0.781007
3,0.389300,0.442384,0.784898
4,0.366500,0.446382,0.786003


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.j

TrainOutput(global_step=3060, training_loss=0.429674505096635, metrics={'train_runtime': 1924.6582, 'train_samples_per_second': 25.438, 'train_steps_per_second': 1.59, 'total_flos': 1.28819172704256e+16, 'train_loss': 0.429674505096635, 'epoch': 4.0})

# Evaluation


In [ ]:
print("Prediction on validation data:")
print(trainer.predict(tokenized_ds['val'])[-1])

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 32


Prediction on validation data:


{'test_loss': 0.4463815689086914, 'test_f1': 0.7860028387378535, 'test_runtime': 12.5404, 'test_samples_per_second': 79.743, 'test_steps_per_second': 2.552}


In [ ]:
print("Prediction on test data:")
print(trainer.predict(tokenized_ds['test'])[-1])

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 32


Prediction on test data:


{'test_loss': 0.37241330742836, 'test_f1': 0.8194225721784778, 'test_runtime': 11.3094, 'test_samples_per_second': 76.043, 'test_steps_per_second': 2.387}


# Model summary

In [ ]:
print("model config")
print(model.config)
print("model summary")
print(summary(model))

model config
BertConfig {
  "_name_or_path": "GroNLP/hateBERT",
  "_num_labels": 2,
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

model summary
Layer (type:depth-idx)                                  Param #
BertForSequenceClassification                           --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                    

# Pushing and saving output

In [ ]:
id2label = {cl.str2int(i): i for i in cl.names}
label2id = {v: k for k, v in id2label.items()}
id2label

{0: 'OFF', 1: 'NOT'}

In [ ]:
model.config.update({
    'id2label': id2label,
    'label2id': label2id,
})
model.config

BertConfig {
  "_name_or_path": "GroNLP/hateBERT",
  "_num_labels": 2,
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "OFF",
    "1": "NOT"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NOT": 1,
    "OFF": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.create_repo(repo_id="hateBert-finetuned-0", private=True)

'https://huggingface.co/sara-nabhani/hateBert-fintuned-0'

In [ ]:
hub_name = 'sara-nabhani/hateBert-finetuned-0'
tokenizer.push_to_hub(hub_name)
model.push_to_hub(hub_name)

tokenizer config file saved in /tmp/tmp6rhc2p5x/tokenizer_config.json
Special tokens file saved in /tmp/tmp6rhc2p5x/special_tokens_map.json
Uploading the following files to sara-nabhani/hateBert-fintuned-0: vocab.txt,tokenizer_config.json,special_tokens_map.json,tokenizer.json
Configuration saved in /tmp/tmpgs_tjo5g/config.json
Model weights saved in /tmp/tmpgs_tjo5g/pytorch_model.bin
Uploading the following files to sara-nabhani/hateBert-fintuned-0: pytorch_model.bin,config.json


CommitInfo(commit_url='https://huggingface.co/sara-nabhani/hateBert-fintuned-0/commit/9e97432b830ba91e714cd217e02cf3099bf372d7', commit_message='Upload BertForSequenceClassification', commit_description='', oid='9e97432b830ba91e714cd217e02cf3099bf372d7', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
val_predictions = np.argmax(trainer.predict(tokenized_ds['val'])[0], axis=-1)
test_predictions = np.argmax(trainer.predict(tokenized_ds['test'])[0], axis=-1)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 32


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 32


In [ ]:
import pandas as pd
val_data_frame=pd.DataFrame({'tweet': ds['val']['tweet'], 'true_label': ds['val']['label'], 'pred_label': val_predictions})
test_data_frame=pd.DataFrame({'tweet': ds['test']['tweet'], 'true_label': ds['test']['label'], 'pred_label': test_predictions})

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
test_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/models-output/hateBert-finetuned-0.csv', index=False)
val_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/val-models-output/hateBert-finetuned-0.csv', index=False)